## from_dicts() bs
How to handle inconsistent schemas in regards to input from nosql databases such as elastic?

Probably not going to use this code.
The problem is that what if you write a detection and the fields change per query?
But you're writing the detection for a specific set of logs that probably won't change schema.
Think I'm going to move to a multi-dataframe format

In [2]:
import polars as pl
import json
import logging

# Simple examples
a = [
    {
        "log": {
            "file": {
                "path": "/nsm/import/acb5b3eacd804c4c17d25fb312581610/suricata/eve-2022-11-10-07:20.json"
            },
            "offset": 9491215,
            "id": {
                "uid": "1017858934534263",
                "resp_fuids": None
            }
        }
    }
]

b = [
    {  
        "log": {
            "file": {
                "path": "/nsm/import/acb5b3eacd804c4c17d25fb312581610/zeek/logs/files.log"
            },
            "offset": 7559607,
            "id": {
                "uid": [
                    "CAnltOiClThlO6ZFk"
                ],
                "fuid": "FdZyiY2Kf5yf3L4239"
            }
        }
    }
]

dfa = pl.from_dicts(a)
dfb = pl.from_dicts(b)
dfc = pl.from_dicts(a+b)

In [38]:
priorities = {
    'list': {
        'name': 'list',
        'priority': 5,
        'super': set(),
    },
    'str': {
        'name': 'str',
        'priority': 4,
        'super': {'list'}
    },
    'float': {
        'name': 'float',
        'priority': 3,
        'super': {'str', 'list'}
    },
    'int': {
        'name': 'int',
        'priority': 2,
        'super': {'float', 'str', 'list'}
    },
    'bool': {
        'name': 'bool',
        'priority': 1,
        'super': {'str', 'list'}
    },
}

types = {
    'list': list,
    'str': str,
    'float': float,
    'int': int,
    'bool': bool
}

# Takes in two schemas and figures it out
# Should be an end type, that is in the priorities dict
def resolve_conflict(a, b, sorted=False):
    aname = type(a).__name__
    pa = priorities[aname]
    bname = type(b).__name__
    pb = priorities[bname]
    
    if aname == bname:
        return a
    
    if not sorted:
        if pa['priority'] > pb['priority']:
            return resolve_conflict(a, b, sorted=True)
        else:
            return resolve_conflict(b, a, sorted=True)
        
    # At this point a should be a higher priority than b
    
    # See if we can super b
    if aname in pb['super']:
        if aname == 'list':
            c = resolve_conflict(a[0], b)
            return [c]
        
        return a
    # Find a common parent
    # Likely either a str or list
    else:
        s = tuple(pa['super'] & pb['super'])
        
        if len(s) == 0:
            logging.warning(f'Could not find a sutable agreement for types {aname} > {bname} when merging, defaulting to str...')
            return str
        
        if s[0] != 'list':
            return type[s[0]]
        
        raise Exception(f'Could only raise types {aname} > {bname} to list')

def gen_schema(data:dict):
    schema = {}
    for i in data:
        itype = type(data[i])
        
        if itype == dict:
            schema[i] = gen_schema(data[i])
        elif itype == list:
            schema[i] = [type(data[i][0])]
        else:
            schema[i] = itype
        
    return schema

def merge_schema(a:dict, b:dict):
    if not a:
        return a
    
    if not b:
        return b
    
    c = {}
    for i in a:
        if i in b:
            ta = type(a[i])
            tb = type(b[i])
            
            if ta == tb and dict not in (ta, tb):
                c[i] = ta
            elif ta == tb and dict in (ta, tb):
                # yeah you can only compromise between a dict and anything as stringing both
                # and for that reason I'm out
                pass

def gen_super_schema(data:list[dict]):
    ss = {}
    for i in data:
        s = gen_schema(i)
        
        if s != ss:
            print('Merging')
            ss = merge_schema(s, ss)
            

#pl.concat([dfa, dfb], how="diagonal_relaxed")

#print('what the feds want you to believe')
#print(json.dumps(dfc.to_dicts(), indent=2))

gen_super_schema(a+a+b+a+b)

Merging
Merging
Merging
Merging
Merging


Using the schema from elastic itself

In [36]:
import json
import polars as pl

with open('./so-network-index.json') as f:
    index = json.loads(f.read())
    
with open('./so-network-data.json') as f:
    json_data = json.loads(f.read())
    
props = index['so-network-2022.10']['mappings']['properties']

def gen_schema(props:dict):
    schema = {}
    for i in props:
        if 'properties' in props[i]:
            schema[i] = pl.Struct(gen_schema(props[i]['properties']))
            continue
        
        ptype = props[i]['type']
        if ptype in ('date', 'date_nanos'):
            schema[i] = pl.String
        elif ptype in ('keyword', 'text', 'constant_keyword', 'ip', 'binary',
                       'wildcard', 'match_only_text'):
            schema[i] = pl.String
        elif ptype in ('long', 'scaled_float'):
            schema[i] = pl.Int64
        elif ptype in ('integer'):
            schema[i] = pl.Int32
        elif ptype in ('short'):
            schema[i] = pl.Int16
        elif ptype in ('byte'):
            schema[i] = pl.Int8
        elif ptype in ('double'):
            schema[i] = pl.Float64
        elif ptype in ('float', 'half_float'):
            schema[i] = pl.Float32
        elif ptype in ('unsigned_long'):
            schema[i] = pl.UInt64
        elif ptype in ('boolean'):
            schema[i] = pl.Boolean
        elif ptype in ('object', 'flattened', 'nested'):
            schema[i] = pl.String
        elif ptype in ('geo_point'):
            schema[i] = pl.String
        # elif ptype in ('')
        else:
            print(f'{i} {ptype}')

    return schema

schema = pl.Schema(gen_schema(props))
unnested = [x['_source'] for x in json_data]
data = pl.from_dicts(unnested, schema=schema, infer_schema_length=4096)
ddicts = data.to_dicts()

for i in ddicts:
    if 'client' in i:
        print(i['client'])

ComputeError: could not append value: ["import", "beats_input_codec_plain_applied"] of type: list[str] to the builder; make sure that all rows have the same schema or consider increasing `infer_schema_length`

it might also be that a value overflows the data-type's capacity